# Análise da distribuição dos locais de tratamento

Nesse notebook iremos analisar a dustribuição dos locais de tratamento dentro de certos estados e vamos comparar com outros estados com população parecida. 

In [177]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

from geopy.distance import geodesic

%matplotlib inline
import mplleaflet
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.pylab as pylab
import seaborn as sns

import os
import warnings

from k_medoids import KMedoidsWeighted
import helper as hp

warnings.simplefilter("ignore")
tqdm.pandas()

np.random.seed(3)

In [178]:
data_dir = '../data/'
quimio_name = 'Pulmão Quimioterapia SIA-SUS.csv'
radio_name = 'Pulmão Radioterapia SIA-SUS.csv'
mortalidade_name = 'Mortalidade Pulmão SIM-SUS.csv'
municipios_name = 'dados_municipios.csv'

usecols_quimio = ['AP_VL_AP', 'AP_UFMUN', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 
               'AQ_ESTADI', 'AQ_DTIDEN', 'AQ_DTINTR']

usecols_radio = ['AP_VL_AP', 'AP_UFMUN', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 
               'AR_ESTADI', 'AR_DTIDEN', 'AR_DTINTR']

usecols_mortalidade = ['DTOBITO', 'IDADE', 'SEXO', 'RACACOR', 'CODMUNRES', 'LOCOCOR', 'CODESTAB',
                      'CODMUNOCOR', 'CAUSABAS']

dtype_cols = {'AP_MUNPCN': str, 'AP_UFMUN': str}

codigo_uf = '42' #estado a ser analisado
nome_uf = 'SantaCatarina'

In [179]:
# Lendo os dataframes:

quimio = pd.read_csv(os.path.join(data_dir, quimio_name), encoding='latin', 
                     error_bad_lines=False, low_memory=False, usecols=usecols_quimio, dtype=dtype_cols)

radio = pd.read_csv(os.path.join(data_dir, radio_name), encoding='latin',
                    error_bad_lines=False, low_memory=False, usecols=usecols_radio, dtype=dtype_cols)

mortalidade = pd.read_csv(os.path.join(data_dir, mortalidade_name), encoding='latin',
                         error_bad_lines=False, low_memory=False, usecols=usecols_mortalidade)

municipios = pd.read_csv(os.path.join(data_dir, municipios_name), index_col=0, dtype={'COD_MUN': str})

In [180]:
# Removendo o último digito para se adequar ao dataframe das APACs:

municipios.index = municipios['COD_MUN'].apply(lambda x: x[:-1])

In [181]:
quimio['distância'] = quimio.progress_apply(lambda row: hp.calculte_distance(row, municipios), axis=1)

100%|██████████| 279825/279825 [00:58<00:00, 4794.27it/s]


Primeiramente vamos analisar apenas os pacientes de quimioterapia que moram e se trataram no estado em questão.

In [182]:
df_estado = hp.generate_df_estado(quimio, municipios, codigo_uf)

In [183]:
# Plotando onde os pacientes residem e onde foram atendidos para um estado qualquer:
fig, ax = plt.subplots(figsize=[10,10])
ax = sns.scatterplot(data=df_estado.groupby(['LON_PCN', 'LAT_PCN']).size().reset_index(), x='LON_PCN', y='LAT_PCN', color='b', ax=ax)
ax = sns.scatterplot(data=df_estado.groupby(['LON_MUN', 'LAT_MUN']).size().reset_index(), x='LON_MUN', y='LAT_MUN', color='r', ax=ax, s=200)
mplleaflet.display(fig=fig)

Os pontos em azul são todas as cidades onde houve um caso de paciente com câncer e os pontos em vermelho são as possíveis cidades de tratamento.

# Claramente, o número de locais capazes de tratar os pacientes é insuficiente para tratar toda a população do estado.

In [184]:
fig, ax = plt.subplots(figsize=[15,15])

#Onde as pessoas daquela cidade foram se tratar
temp_df = pd.crosstab(index=df_estado['AP_MUNPCN'], columns=df_estado['AP_UFMUN'])
#Quantidade de ocorrencias na cidade
tamanho_municipio = df_estado['AP_MUNPCN'].value_counts()

norm = mpl.colors.Normalize(vmin=0, vmax=len(temp_df.columns))
cmap = cm.Dark2
m = cm.ScalarMappable(norm=norm, cmap=cmap)
colors = {col: m.to_rgba(i) for i, col in enumerate(temp_df.columns)}

plt.figure(figsize=[12,12])
for mun in tqdm(temp_df.index):
    lat, long = hp.coord_from_codigo(mun, municipios)
    if lat==0:
        continue
        
    ax.scatter(long, lat, s=tamanho_municipio.loc[mun], color=colors[temp_df.loc[mun].argmax()])

mplleaflet.display(fig=fig)

100%|██████████| 258/258 [00:00<00:00, 367.60it/s]


<Figure size 864x864 with 0 Axes>

Vamos calcular a distância média que uma pessoa de outra cidade tem que percorre para se tratar na condição atual. E quantas pessoas são atendidas em cada centro de atendimento

In [185]:
print("Distância média percorrida por pessoas que não estão na cidade de tratamento: {:.2f}km".format(df_estado[df_estado['distância']>0]['distância'].mean()))

Distância média percorrida por pessoas que não estão na cidade de tratamento: 38.34km


In [186]:
for nome in temp_df.columns:
    print("{} --> {} pacientes".format(municipios.loc[nome]['NOME'], temp_df.sum().loc[nome]))

Blumenau --> 1562 pacientes
Chapecó --> 1592 pacientes
Criciúma --> 2096 pacientes
Florianópolis --> 2264 pacientes
Itajaí --> 1135 pacientes
Jaraguá do Sul --> 721 pacientes
Joaçaba --> 1521 pacientes
Joinville --> 1585 pacientes
Lages --> 945 pacientes
Porto União --> 229 pacientes
Rio do Sul --> 32 pacientes
São Bento do Sul --> 16 pacientes
Tubarão --> 697 pacientes


## Propondo novos centros de tratamento

Como podemos notar pelos mapas, há, na maioria dos estados brasileiros, uma má distribuição dos locais de tratamento em relação a população. Dessa forma, criamos uma nova implementação do algoritimo K medoids para lidar com pesos nos pontos de entrada, a classe está implementada no arquivo k_medoids.py

In [187]:
X = [list(hp.coord_from_codigo(codigo, municipios)) for codigo in df_estado['AP_MUNPCN'].value_counts().index]
weights = df_estado['AP_MUNPCN'].value_counts()

In [188]:
X_weighted = [x+[w] for x,w in zip(X, weights)]

In [189]:
#quantidade inicial de centros de tratamento
n_tratamento = len(temp_df.columns)
#quantidacde de novos centros
n_novos = 1
clustering = KMedoidsWeighted(n_clusters=n_tratamento+n_novos)

In [190]:
medoids_idxs, clusters = clustering.fit(X_weighted, max_itr=10, verbose=1)

Iteration: 0 ... loss: 6453.62280691503
Iteration: 1 ... loss: 2466.5822927636673
Iteration: 2 ... loss: 2141.2356307010655
Iteration: 3 ... loss: 2137.202535727609


In [191]:
fig, ax = plt.subplots(figsize=[15,15])
norm = mpl.colors.Normalize(vmin=0, vmax=len(medoids_idxs))
cmap = cm.Dark2
m = cm.ScalarMappable(norm=norm, cmap=cmap)

for i,cluster in enumerate(clusters):
    for p in cluster:
        if p in medoids_idxs:
            plt.scatter(X[p][1], X[p][0], color=m.to_rgba(i), s=400)
        else:
            plt.scatter(X[p][1], X[p][0], color=m.to_rgba(i))
        
mplleaflet.display(fig=fig)

In [192]:
tot_dist = 0
tot_pop = 0
for medoid, cluster in zip(medoids_idxs, clusters):
    for p in cluster:
        if p!=medoid:
            pop = X_weighted[p][2]
            lat1, long1 = X_weighted[p][0], X_weighted[p][1]
            lat2, long2 = X_weighted[medoid][0], X_weighted[medoid][1]
            tot_dist += geodesic((lat1, long1),(lat2, long2)).km*pop
            tot_pop += pop
        
mean_new_dist = tot_dist/tot_pop

In [193]:
print("Distância média percorrida por pessoas que não estão na cidade de tratamento com a nova configuração: {:.2f}km".format(mean_new_dist))

Distância média percorrida por pessoas que não estão na cidade de tratamento com a nova configuração: 26.50km
